### Joblib for parallel processing
https://joblib.readthedocs.io/en/latest/

Many scikit learn models which support an n_jobs parameter implement joblib at the back to run the jobs concurrently

In [17]:
!sysctl hw.physicalcpu hw.logicalcpu

hw.physicalcpu: 2
hw.logicalcpu: 4


In [18]:
import time 
from joblib import Parallel, delayed

In [19]:
def work(arg):    
    
    print ("Argument received", arg)
    i, j = arg    

    time.sleep(5)    
    print ("Data merged %s_%s" % (i, j))

    return "%s_%s" % (i, j)

In [20]:
return_val = work((2, 3))

Argument received (2, 3)
Data merged 2_3


In [21]:
return_val

'2_3'

#### The delayed() function
When used with a map function, the delayed() function initializes the function call with the corresponding arguments without actually executing it. This allows the function call to be set up and then executed concurrently using the joblib.Parallel instance

When called independently, the function will execute.

In [23]:
return_val = delayed(work((2, 3)))



Argument received (2, 3)
Data merged 2_3


In [24]:
return_val

<function joblib.parallel.delayed.<locals>.delayed_function>

In [8]:
type(return_val)

function

In [9]:
return_val()

('2_3', (), {})

#### The Parallel object
This will create a pool of workers which could be a Thread pool or a Process pool, depending on the value of the "backend" argument. 

In [25]:
Parallel(n_jobs=1, verbose=1, backend="threading")

Parallel(n_jobs=1)

#### Specify a set of arguments for concurrent instances of the work function
Up to 4 concurrent calls can be invoked

In [26]:
arg_instances = [(1, 1), (1, 2), (1, 3), (1, 4)]

#### Use the map function
This accepts two arguments:
* the function to execute in each concurrent thread/process
* an iterable of arguments for each function call


In [27]:
start_time = time.time()

Parallel(n_jobs=1, 
         verbose=1, 
         backend="threading")(map(delayed(work), 
                                  arg_instances))

total_time = time.time() - start_time

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Argument received (1, 1)
Data merged 1_1
Argument received (1, 2)
Data merged 1_2
Argument received (1, 3)
Data merged 1_3
Argument received (1, 4)
Data merged 1_4


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   20.0s finished


In [28]:
print('Total time: ', total_time)

Total time:  20.01704692840576


In [29]:
start_time = time.time()

Parallel(n_jobs=2, 
         verbose=1, 
         backend="threading")(map(delayed(work), 
                                  arg_instances))

total_time = time.time() - start_time
print('Total time: ', total_time)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Argument receivedArgument received (1, 2)
 (1, 1)
Data merged 1_2Data merged 1_1
Argument received (1, 3)

Argument received (1, 4)
Data merged 1_3Data merged 1_4

Total time:  10.023071050643921


[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:   10.0s finished


In [30]:
start_time = time.time()

Parallel(n_jobs=4, 
         verbose=1, 
         backend="threading")(map(delayed(work), 
                                  arg_instances))

total_time = time.time() - start_time
print('Total time: ', total_time)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


Argument receivedArgument receivedArgument received Argument received (1, 2)  (1, 4)(1, 1)
(1, 3)


Data merged 1_4Data merged 1_1

Data merged 1_2Data merged 1_3

Total time:  5.019229888916016


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    5.0s remaining:    5.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    5.0s finished


#### Use a multiprocessing backend
This will create a process pool using multiprocessing.Pool. Processes are more heavyweight than threads, but each has its specific use case:
* multithreading: Better for tasks which are I/O or network bound
* multiprocessing: Better for CPU bound tasks

In [31]:
start_time = time.time()

Parallel(n_jobs=3, 
         verbose=1, 
         backend="multiprocessing")(map(delayed(work), 
                                    arg_instances))

total_time = time.time() - start_time
print('Total time: ', total_time)

[Parallel(n_jobs=3)]: Using backend MultiprocessingBackend with 3 concurrent workers.


KeyboardInterrupt: 

In [32]:
start_time = time.time()

Parallel(n_jobs=4, 
         verbose=1, 
         backend="multiprocessing")(map(delayed(work), 
                                    arg_instances))

total_time = time.time() - start_time
print('Total time: ', total_time)

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.


KeyboardInterrupt: 